# Importing modules and files

In [2]:
import importlib as imp

import os
print(os.getcwd())
import sys
sys.path.append(os.path.abspath('../../app'))

import preprocessor2 as preprocessing
from preprocessor2 import DataLoader

from keras import backend as K

import numpy as np
import joblib

import tensorflow as tf
from tensorflow import keras
from keras import layers

c:\Users\LiubomyrMaievskyi\source\repos\other\FraudDetectionSystem\ml-system\data_analysis\jupyter


In [3]:
dataLoader = DataLoader(2048)

dataset_train = dataLoader.load_dataset(path='../datasets/fraudTrain.csv',
                                  preprocess=True, to_process=10000)
dataset_test = dataLoader.load_dataset(path='../datasets/fraudTest.csv',
                                  preprocess=True, to_process=10000)

KeyboardInterrupt: 

# Building tensorflow model

In [ ]:
def masked_loss(label, pred):
  mask = label != 2
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= (label == 1).numpy().astype(int) * 1.5 + (label == 0).numpy().astype(int)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 2

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [ ]:
def masked_precision(label, pred):
    pred = tf.argmax(pred, axis=2)
    label = tf.cast(label, pred.dtype)

    mask = label != 2

    true_positive = tf.math.logical_and(1 == pred, mask)
    predicted_positive = tf.math.logical_and(1 == pred, 1 == label)

    true_positive = tf.cast(true_positive, dtype=tf.float32)
    predicted_positive = tf.cast(predicted_positive, dtype=tf.float32)
    precision = (tf.reduce_sum(predicted_positive) + 1) / (tf.reduce_sum(true_positive) + 1)

    return precision

In [ ]:
def masked_recall(label, pred):
    pred = tf.argmax(pred, axis=2)
    label = tf.cast(label, pred.dtype)

    mask = label != 2

    true_positive = tf.math.logical_and(1 == label, mask)
    predicted_positive = tf.math.logical_and(1 == pred, 1 == label)

    true_positive = tf.cast(true_positive, dtype=tf.float32)
    predicted_positive = tf.cast(predicted_positive, dtype=tf.float32)
    precision = (tf.reduce_sum(predicted_positive) + 1) / (tf.reduce_sum(true_positive) + 1)

    return precision

In [ ]:
_EPSILON = 10e-8

value = 1

def recall(y_true, y_pred, c=value):
    pred_c = K.cast(K.equal(K.argmax(y_pred, axis=-1), c), K.floatx())
    true_c = K.cast(K.equal(y_true, c), K.floatx())
    true_positives = K.sum(pred_c * true_c)
    possible_postives = K.sum(true_c)
    return true_positives / (possible_postives + _EPSILON)


def precision(y_true, y_pred, c=value):
    pred_c = K.cast(K.equal(K.argmax(y_pred, axis=-1), c), K.floatx())
    true_c = K.cast(K.equal(y_true, c), K.floatx())
    true_positives = K.sum(pred_c * true_c)
    pred_positives = K.sum(pred_c)
    return true_positives / (pred_positives + _EPSILON)

In [ ]:
sample_weight = np.ones(shape=(len(dataset_train),))
sample_weight[dataset_train == 1] = 299.0

In [ ]:
def get_model_LSTM(shape=None):
    input = layers.Input(shape=(shape[0], shape[1], ), name='example')

    x = tf.keras.layers.Permute((1, 2))(input)
    x = layers.Dense(128, activation='tanh')(x)
    x = layers.LSTM(128, return_sequences=True)(x)
    x = layers.Dense(256, activation='tanh')(x)
    x = layers.LSTM(256, return_sequences=True)(x)
    
    output = layers.Dense(3, activation='softmax')(x)

    model = keras.models.Model(
        inputs=input, outputs=output, name='fraud-detector'
    )

    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate=0.001),
        loss = masked_loss, #tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics = ['accuracy', masked_accuracy, masked_precision, masked_recall],
        run_eagerly=True
    )

    return model

In [ ]:
model = get_model_LSTM(shape=(32, 26))
model.summary()

Model: "fraud-detector"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 example (InputLayer)        [(None, 32, 26)]          0         
                                                                 
 permute (Permute)           (None, 32, 26)            0         
                                                                 
 dense (Dense)               (None, 32, 128)           3456      
                                                                 
 lstm (LSTM)                 (None, 32, 128)           131584    
                                                                 
 dense_1 (Dense)             (None, 32, 256)           33024     
                                                                 
 lstm_1 (LSTM)               (None, 32, 256)           525312    
                                                                 
 dense_2 (Dense)             (None, 32, 3)          

In [ ]:
model.fit(
        dataset_train,
        batch_size=2048,
        epochs=100,
        validation_data = dataset_test
)

Epoch 1/100


c:\Users\LiubomyrMaievskyi\source\repos\other\FraudDetectionSystem\ml-system\.venv\Lib\site-packages\keras\src\backend.py:5714: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


5/5 [==============================] - 24s 5s/step - loss: 0.4063 - accuracy: 0.2739 - masked_accuracy: 0.9937 - masked_precision: 1.0000 - masked_recall: 0.4041 - val_loss: 0.0606 - val_accuracy: 0.2550 - val_masked_accuracy: 0.9989 - val_masked_precision: 1.0000 - val_masked_recall: 0.4381
Epoch 2/100
5/5 [==============================] - 22s 5s/step - loss: 0.1073 - accuracy: 0.2477 - masked_accuracy: 0.9937 - masked_precision: 1.0000 - masked_recall: 0.4041 - val_loss: 0.0204 - val_accuracy: 0.2550 - val_masked_accuracy: 0.9989 - val_masked_precision: 1.0000 - val_masked_recall: 0.4381
Epoch 3/100
5/5 [==============================] - 23s 5s/step - loss: 0.0801 - accuracy: 0.2477 - masked_accuracy: 0.9937 - masked_precision: 1.0000 - masked_recall: 0.4041 - val_loss: 0.0151 - val_accuracy: 0.2550 - val_masked_accuracy: 0.9989 - val_masked_precision: 1.0000 - val_masked_recall: 0.4381
Epoch 4/100
5/5 [==============================] - 25s 5s/step - loss: 0.0619 - accuracy: 0.2477 

KeyboardInterrupt: 

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        positions = tf.range(start=0, limit=self.maxlen, delta=1)
        positions = self.pos_emb(positions)
        return x + positions
        
class BaseAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

class LogisticEndpoint(keras.layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
        self.accuracy_fn = keras.metrics.BinaryAccuracy(name="accuracy")

    def call(self, logits, targets=None, sample_weight=None):
        if targets is not None:
            # Compute the training-time loss value and add it
            # to the layer using `self.add_loss()`.
            loss = self.loss_fn(targets, logits, sample_weight)
            self.add_loss(loss)

            # Log the accuracy as a metric (we could log arbitrary metrics,
            # including different metrics for training and inference.
            self.add_metric(self.accuracy_fn(targets, logits, sample_weight))

        # Return the inference-time prediction tensor (for `.predict()`).
        return tf.nn.softmax(logits)
        

class CausalSelfAttention(BaseAttention):
    def call(self, q, v):
        attn_output = self.mha(
            query=q,
            value=v,
            key=v,
            use_causal_mask = True)
        x = self.add([v, attn_output])
        x = self.layernorm(x)
        return x


class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = CausalSelfAttention(num_heads=num_heads,
                                             key_dim=embed_dim)
        self.ffn = keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),
            layers.Dropout(rate)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)                      # self-attention layer
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)                # layer norm
        ffn_output = self.ffn(out1)                                 # feed-forward layer
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)                   # layer norm

In [ ]:
def get_TRANSFORMER(shape=(32, 20)):
    input = layers.Input(name="example", shape=(shape[0], shape[1]))
    x = tf.keras.layers.Normalization(axis = 1)(input)
    
    x = layers.Dense(64, activation='relu')(x)
    x = TokenAndPositionEmbedding(maxlen=32, vocab_size=3, embed_dim=64)(x)
    x = TransformerBlock(64, 5, 128)(x)
    x = TransformerBlock(64, 5, 128)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = TransformerBlock(32, 5, 128)(x)
    x = layers.Dense(32, activation='relu')(x)
    output = layers.Dense(3, activation='softmax', name='finish')(x)

    model = tf.keras.models.Model(
        inputs=input, outputs=output, name="fraud_detector"
    )
    return model

In [ ]:
transformer = get_TRANSFORMER(shape=(32, 26))

transformer.compile(
        optimizer = keras.optimizers.Adam(learning_rate=0.001),
        loss = masked_loss,
        metrics = ['accuracy', masked_accuracy, masked_precision, masked_recall],
        run_eagerly=True
)

transformer.fit(
    dataset_train,
    batch_size=64,
    epochs = 100,
    validation_data=dataset_test
)

Epoch 1/100


/usr/local/lib/python3.9/dist-packages/keras/backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


5/5 [==============================] - 8s 373ms/step - loss: 2.5908 - accuracy: 0.1018 - masked_accuracy: 0.3906 - masked_precision: 0.0269 - masked_recall: 0.6303 - val_loss: 1.8862 - val_accuracy: 0.0016 - val_masked_accuracy: 0.0063 - val_masked_precision: 0.0064 - val_masked_recall: 1.0000
Epoch 2/100
5/5 [==============================] - 2s 359ms/step - loss: 1.9589 - accuracy: 0.0363 - masked_accuracy: 0.1435 - masked_precision: 0.0062 - masked_recall: 0.8865 - val_loss: 1.7935 - val_accuracy: 0.0016 - val_masked_accuracy: 0.0063 - val_masked_precision: 0.0064 - val_masked_recall: 1.0000
Epoch 3/100
5/5 [==============================] - 2s 369ms/step - loss: 1.8650 - accuracy: 0.0152 - masked_accuracy: 0.0597 - masked_precision: 0.0063 - masked_recall: 0.9611 - val_loss: 1.7749 - val_accuracy: 0.0016 - val_masked_accuracy: 0.0063 - val_masked_precision: 0.0064 - val_masked_recall: 1.0000
Epoch 4/100
5/5 [==============================] - 2s 362ms/step - loss: 1.8051 - accuracy:

KeyboardInterrupt: ignored